We use 10,20,30 to represent S,M,L.

In [1]:
import numpy as np
# from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
import matplotlib.pyplot as plt

In [2]:
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))


def print_error_rate(err):
    print('Error rate: Training: %.4f - Test: %.4f' % err)

In [3]:
class DecisionStump():
    def __init__(self):
        self.best_stump={}
    def _classify(self, X, dim, threshold, inequal):
        y = np.ones(X.shape[0])
        if inequal == 'lt':
            y[X[:, dim] <= threshold] = -1.
        else:
            y[X[:, dim] > threshold] = -1.
        return y

    def fit(self, X, Y, W, num_step = 10):
        n, dim = X.shape
        min_err = np.inf
        for d in range(dim):
            range_min = min(X[:,d])
            range_max = max(X[:,d])
            step_size = (range_max - range_min)/num_step
            for i in range(0, num_step+1):
                for inequal in ['lt', 'gt']:
                    threshold = range_min + i * step_size
                    pred = self._classify(X, d, threshold, inequal)
                    err = np.ones(n)
                    # print(err.shape)
                    err[pred == Y] = 0
                    err = np.dot(err,W)
                    # print(err.shape)
                    if err < min_err:
                        min_err = err
                        # bestClassEst = predicatedVal.copy()
                        self.best_stump['dim'] = d
                        self.best_stump['threshold'] = threshold
                        self.best_stump['ineq'] = inequal
    def predict(self, X):
        assert self.best_stump, "Please call fit() first"
        return self._classify(X, self.best_stump['dim'], self.best_stump['threshold'], self.best_stump['ineq'])

In [4]:
def train(X,Y,num_iterations):

    weak_models = []
    alphas = []

    N, _ = X.shape
    w = np.ones(N) / N

    for t in range(num_iterations):
        model = DecisionStump()

        model.fit(X, Y, w)
        pred = model.predict(X)
        # print("pred : ",pred)
        err = w.dot(np.sign(pred) != Y)
        # print("err : ",err)
        alpha = (np.log(1 - err) - np.log(err)) / 2

        w = w * np.exp(- alpha * Y * pred)
        w = w / w.sum()
        print("iter {} : alpha: {}  err: {} ".format(t,alpha,err))
        weak_models.append(model)
        alphas.append(alpha)
    return {'weak_models':weak_models,'alphas':alphas}

In [5]:
def validate(model,X,Y=None):
    N, _ = X.shape
    y = np.zeros(N)
    for (h, alpha) in zip(model['weak_models'], model['alphas']):
        # print("alpha = ",alpha)
        y = y + alpha * h.predict(X)
    y = np.sign(y)
    if Y is None:
        return list(map(int,y))
    else:
        # print("y : ",y )
        print("error_rate : ",get_error_rate(y,Y))

In [6]:
if __name__ == '__main__':

    data_x = []
    data_y = []
    with open('data.txt', 'r') as f:
        lines = f.readlines()
        line_num = len(lines)
        for line in lines:
            line = line.strip()
            x = [float(line.split(',')[0]), float(line.split(',')[1])]
            y = int(line.split(',')[2])
            data_x.append(x)
            data_y.append(y)
        # print(line_num)
    X_train = np.array(data_x)
    Y_train = np.array(data_y)
    # print(X_train.shape, Y_train.shape)
    X_val, Y_val = X_train, Y_train
    X_test = np.array([[1.,20.]])

    model = train(X_train,Y_train,50)

    validate(model,X_val,Y_val)

    print(validate(model,X_test))

iter 0 : alpha: 0.49041462650586315  err: 0.2727272727272727 
iter 1 : alpha: 0.44365159750045136  err: 0.2916666666666667 
iter 2 : alpha: 0.2711621454126808  err: 0.3676470588235294 
iter 3 : alpha: 0.21244159698263287  err: 0.39534883720930236 
iter 4 : alpha: 0.1748368742398742  err: 0.41346153846153855 
iter 5 : alpha: 0.14862576173396574  err: 0.42622950819672134 
iter 6 : alpha: 0.1707841442090534  err: 0.41542857142857154 
iter 7 : alpha: 0.08490116489746746  err: 0.4576511218588691 
iter 8 : alpha: 0.14364218953406716  err: 0.42866882563490705 
iter 9 : alpha: 0.12550568463839373  err: 0.43757458233890206 
iter 10 : alpha: 0.11145242967253077  err: 0.4445033815143879 
iter 11 : alpha: 0.10023889595063845  err: 0.4500477440916686 
iter 12 : alpha: 0.10549372686541791  err: 0.4474479414128399 
iter 13 : alpha: 0.08582675191323724  err: 0.4571916844462259 
iter 14 : alpha: 0.09376728298990844  err: 0.4532532820420938 
iter 15 : alpha: 0.08570902780493045  err: 0.45725011561797324

The model cannot converge.
The label of (1,M) is 1.